In [1]:
import pandas as pd
import utils
import matplotlib.pyplot as plt
import random
import plotly.express as px
import seaborn as sns
import numpy as np

random.seed(9000)

plt.style.use("seaborn-ticks")
plt.rcParams["image.cmap"] = "Set1"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Set1.colors)
%matplotlib inline

In [2]:
n_samples = 1000
n_replicates = 4  #number of sample replicates within each plate 
metadata_common = 'Metadata_moa'
metadata_perturbation = 'Metadata_broad_sample'
group_by_feature = 'Metadata_broad_sample'
#batch = "Scope1_MolDev_10X"

experiment_df = (
    pd.read_csv('output/experiment-metadata.tsv', sep='\t')
#    .query('Batch==@batch')
)
experiment_df.head(50)

,Vendor,Batch,Plate_Map_Name,Assay_Plate_Barcode,Modality,Binning,Magnification,Number_of_channels,Images_per_well,z_plane,BF_Zplanes,Anomaly
0,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_4site_10XPA,Confocal,1,10,6,4,1,NaN,none
1,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crest,Confocal,1,10,6,4,1,NaN,none
2,MolDev,Scope1_MolDev_10X_4siteZ,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crestz,Confocal,1,10,6,4,3,NaN,none
3,MolDev,Scope1_MolDev_20X_4site,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_20XPA_Crestz,Confocal,1,20,6,4,3,NaN,none
4,MolDev,Scope1_MolDev_20X_9site,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_9site_20XPA,Confocal,1,20,6,9,1,NaN,none
5,MolDev,Scope1_MolDev_20X_9site,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_9site_20XPA_Crest,Confocal,1,20,6,9,1,NaN,none
6,MolDev,Scope1_MolDev_20X_Adaptive,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_Adaptive_20XPA,Confocal,1,20,6,3,1,NaN,none
7,Nikon,Scope1_Nikon_10X,JUMP-MOA_compound_platemap,BR00117060a10x,Widefield,1,10,4,1,1,NaN,none
8,Nikon,Scope1_Nikon_10X,JUMP-MOA_compound_platemap,BR00117061a10x,Widefield,1,10,4,1,1,NaN,none
9,Nikon,Scope1_Nikon_10X,JUMP-MOA_compound_platemap,BR00117062a10x,Widefield,1,10,4,1,1,NaN,none


In [4]:
corr_replicating_df = pd.DataFrame()
corr_matching_df = pd.DataFrame()

for fov in experiment_df["Images_per_well"] 
    for a_vendor in experiment_df["Vendor"].unique():
        vendor_data = experiment_df.loc[experiment_df["Vendor"] == a_vendor]
    #     print([a_vendor, len(vendor_data)])
        for a_batch in vendor_data["Batch"].unique():
            batch_data = vendor_data.loc[vendor_data["Batch"] == a_batch]
    #         print([a_batch, len(batch_data)])
            for a_plate in batch_data["Assay_Plate_Barcode"].unique():
                plate_data = batch_data.loc[batch_data["Assay_Plate_Barcode"] == a_plate]
    #             print([a_plate, len(plate_data)])
                plate_df_4FoV = utils.load_data_4FoV(a_batch, a_plate, "normalized_feature_select_negcon_batch.csv.gz")
                plate_df_4FoV = utils.remove_negcon_empty_wells(plate_df_4FoV)

ValueError: No objects to concatenate